In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import demoji
# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text

In [2]:
conda update tensorflow


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /Users/roshankhatri/opt/anaconda3
  package name: tensorflow



Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_csv(r'../datasets/Task 1/task2_en_training.tsv', sep = '\t')
df

### Average Number of Tweets per User

In [ ]:
sum(list(dict(df['user_id'].value_counts()).values())) / len(list(dict(df['user_id'].value_counts()).values()))

### Added this code to clean the text<br>
need to change text in the dataset<br>

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub(r"#", "", tweet, 0, re.MULTILINE)
    tweet = re.sub(r"rt", "", tweet, 0, re.MULTILINE)
    tweet = tweet.lower()
    regex_mentions = r"@\w+"
    tweet = re.sub(regex_mentions, "", tweet, 0, re.MULTILINE)
#     regex_hash = r"(#[^\s]+)"
#     tweet = re.sub(regex_hash, "", tweet, 0, re.MULTILINE)
#     hash_matched = re.finditer(
#         regex_hash, tweet['full_text'], re.MULTILINE)
    url_regex = r"(http[^\s]+)"
    tweet = re.sub(url_regex, "", tweet, 0, re.MULTILINE)
#     special_char_re = r"[^(?u)\w]+"
#     tweet = re.sub(special_char_re, " ", tweet, 0, re.MULTILINE)
    return tweet

In [ ]:
def remove_emojis(text):
    emoticons_happy = list([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
    ])
    emoticons_sad = list([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
    ])
    all_emoticons = emoticons_happy + emoticons_sad

    emojis = list(demoji.findall(text).keys())
    clean_text = demoji.replace(text, '')

    for emo in all_emoticons:
        if (emo in clean_text):
            clean_text = clean_text.replace(emo, '')
            emojis.append(emo)

    clean_text = clean_text.replace('\n', '', -1)
    
    # using regex to remove urls
    # regex_url = r"https:(.*)[A-Za-z0-9]"
    # clean_text = re.sub(regex_url, '', clean_text, 0, re.MULTILINE)

    return clean_text #, emojis

In [ ]:
# for tweet in df['tweet']:
#     print(tweet)
    
#     print(clean_tweet(remove_emojis(tweet)))
df['tweet'] = df['tweet'].map(lambda tweet: clean_tweet(remove_emojis(tweet)))                                            

### Highly Imbalanced Dataset<br>
We will have to address this.<br>
We can explore creating synthetic data, using TOMEK-Links, etc.

In [ ]:
df['class'].value_counts().plot(kind='bar')

### Addressing Duplicate Tweets<br>
We must remove duplicate rows to avoid redundancy and model overfitting

In [ ]:
len(df.index) - df['tweet'].nunique()

In [ ]:
df['tweet'].value_counts()[:10].plot(kind='barh')

### Tweets belonging to class 1 are 8.9 characters longer on average

In [ ]:
df[df['class'] == 0]['tweet'].apply(len).mean()

In [ ]:
df[df['class'] == 1]['tweet'].apply(len).mean()

In [ ]:
df_0 = df[df['class'] == 0]
df_0.shape

In [ ]:
df_1 = df[df['class'] == 1]
df_1.shape

In [ ]:
df_0_downsampled = df_0.sample(df_1.shape[0])
df_0_downsampled.shape

In [ ]:
df_balanced = pd.concat([df_1,df_0_downsampled])
df_balanced.shape

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
X_train, X_test, y_train, y_test = train_test_split(df_balanced['tweet'],df_balanced['class'],stratify=df_balanced['class'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
bert_encoder = hub.KerasLayer("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")